In [1]:
#!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
#!tar -xf aclImdb_v1.tar.gz
#!rm -r aclImdb/train/unsup
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [2]:
import os, pathlib, shutil, random

if False:
    base_dir = pathlib.Path('aclImdb')
    val_dir = base_dir / 'val'
    train_dir = base_dir / 'train'
    for category in ('neg', 'pos'):
        os.makedirs(val_dir / category)
        files = os.listdir(train_dir / category)
        random.Random(1337).shuffle(files)
        n_val_samples = int(0.2 * len(files))
        val_files = files[-n_val_samples:]
        for filename in val_files:
            shutil.move(train_dir / category / filename, val_dir / category / filename)

In [3]:
from tensorflow import keras
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size
)

val_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/val', batch_size=batch_size
)

test_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/test', batch_size=batch_size
)

2023-09-20 17:46:45.557629: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-20 17:46:45.681652: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-20 17:46:45.682495: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-20 17:46:46.293512: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


2023-09-20 17:46:47.374022: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-20 17:46:47.374326: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Found 25000 files belonging to 2 classes.


In [4]:
for inputs, targets in train_ds:
    print(inputs.shape, inputs.dtype, inputs[0])
    print(targets.shape, targets.dtype, targets[0])
    break

(32,) <dtype: 'string'> tf.Tensor(b"Movies like these are to the originals what Album Oriented Rock stations are to what music used to be like - repetitive, boring, and drained of all the original energy by a committee of corporate drones. I AM glad that Aragorn wasn't typecast as an expectant psycho by this P.O.S. Go back and watch the 1971 version, count the things that would NEVER be included in a modern version, and thank whatever deity you worship that someone somewhere in the distant past had the balls to write and shoot an original concept movie that wasn't based on someone else's ideas, and wasn't passed through a corporate board before it saw the light of day.", shape=(), dtype=string)
(32,) <dtype: 'int32'> tf.Tensor(0, shape=(), dtype=int32)


In [5]:
from keras import layers

text_vectorization = layers.TextVectorization(max_tokens=20000, output_mode='multi_hot')
text_only_train_ds = train_ds.map(lambda x, y: x)
text_vectorization.adapt(text_only_train_ds)

binary_1gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)
binary_1gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)
binary_1gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)

In [6]:
for inputs, targets in binary_1gram_train_ds:
    print(inputs.shape, inputs.dtype, inputs[0])
    print(targets.shape, targets.dtype, targets[0])
    break

(32, 20000) <dtype: 'float32'> tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32)
(32,) <dtype: 'int32'> tf.Tensor(0, shape=(), dtype=int32)


In [7]:
import keras
from keras import layers, activations, optimizers, losses, metrics, callbacks

def get_model(max_tokens=20000, hidden_dim=16):
    inputs = layers.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation=activations.relu)(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation=activations.sigmoid)(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer=optimizers.RMSprop(), loss=losses.BinaryCrossentropy(), metrics=[metrics.BinaryAccuracy()])
    return model

In [8]:
model = get_model()
model.summary()
callback_list = [
    callbacks.ModelCheckpoint('binary_1gram', save_best_only=True)
]

model.fit(binary_1gram_train_ds, epochs=10, validation_data=binary_1gram_val_ds, callbacks=callback_list)

model = keras.models.load_model('binary_1gram')
print(model.evaluate(binary_1gram_test_ds))

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320033 (1.22 MB)
Trainable params: 320033 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
604/625 [===========================>..] - ETA: 0s - loss: 0.4028 - binary_accuracy: 0.8285INFO:tensorflow:Assets written to: binary_1gram/assets


INFO:tensorflow:Assets written to: binary_1gram/assets


625/625 [==============================] - 1s 2ms/step - loss: 0.4001 - binary_accuracy: 0.8302 - val_loss: 0.2825 - val_binary_accuracy: 0.8898
Epoch 2/10
594/625 [===========================>..] - ETA: 0s - loss: 0.2708 - binary_accuracy: 0.9003INFO:tensorflow:Assets written to: binary_1gram/assets


INFO:tensorflow:Assets written to: binary_1gram/assets


625/625 [==============================] - 1s 2ms/step - loss: 0.2716 - binary_accuracy: 0.9000 - val_loss: 0.2777 - val_binary_accuracy: 0.8952
Epoch 3/10
625/625 [==============================] - 1s 1ms/step - loss: 0.2403 - binary_accuracy: 0.9128 - val_loss: 0.2928 - val_binary_accuracy: 0.8914
Epoch 4/10
625/625 [==============================] - 1s 1ms/step - loss: 0.2233 - binary_accuracy: 0.9247 - val_loss: 0.3053 - val_binary_accuracy: 0.8878
Epoch 5/10
625/625 [==============================] - 1s 1ms/step - loss: 0.2185 - binary_accuracy: 0.9294 - val_loss: 0.3260 - val_binary_accuracy: 0.8854
Epoch 6/10
625/625 [==============================] - 1s 1ms/step - loss: 0.2107 - binary_accuracy: 0.9312 - val_loss: 0.3333 - val_binary_accuracy: 0.8862
Epoch 7/10
625/625 [==============================] - 1s 1ms/step - loss: 0.2031 - binary_accuracy: 0.9322 - val_loss: 0.3437 - val_binary_accuracy: 0.8822
Epoch 8/10
625/625 [==============================] - 1s 1ms/step - loss: 0

In [9]:
text_vectorization = layers.TextVectorization(ngrams=2, max_tokens=20000, output_mode='multi_hot')
text_vectorization.adapt(text_only_train_ds)

In [10]:
binary_2gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)
binary_2gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)
binary_2gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)

model = get_model()
callback_list = [
    callbacks.ModelCheckpoint('binary_2gram', save_best_only=True)
]

model.fit(binary_2gram_train_ds, validation_data=binary_2gram_val_ds, epochs=10, callbacks=callback_list)

model = keras.models.load_model('binary_2gram')
print(model.evaluate(binary_2gram_test_ds))

Epoch 1/10
589/625 [===========================>..] - ETA: 0s - loss: 0.3843 - binary_accuracy: 0.8400INFO:tensorflow:Assets written to: binary_2gram/assets


INFO:tensorflow:Assets written to: binary_2gram/assets


625/625 [==============================] - 1s 2ms/step - loss: 0.3790 - binary_accuracy: 0.8430 - val_loss: 0.2621 - val_binary_accuracy: 0.9002
Epoch 2/10
625/625 [==============================] - 1s 1ms/step - loss: 0.2370 - binary_accuracy: 0.9137 - val_loss: 0.2653 - val_binary_accuracy: 0.9028
Epoch 3/10
625/625 [==============================] - 1s 1ms/step - loss: 0.1951 - binary_accuracy: 0.9334 - val_loss: 0.2740 - val_binary_accuracy: 0.9084
Epoch 4/10
625/625 [==============================] - 1s 1ms/step - loss: 0.1808 - binary_accuracy: 0.9416 - val_loss: 0.2884 - val_binary_accuracy: 0.9050
Epoch 5/10
625/625 [==============================] - 1s 1ms/step - loss: 0.1681 - binary_accuracy: 0.9497 - val_loss: 0.3068 - val_binary_accuracy: 0.9026
Epoch 6/10
625/625 [==============================] - 1s 1ms/step - loss: 0.1600 - binary_accuracy: 0.9516 - val_loss: 0.3227 - val_binary_accuracy: 0.9044
Epoch 7/10
625/625 [==============================] - 1s 1ms/step - loss: 0

In [11]:
text_vectorization = layers.TextVectorization(ngrams=2, max_tokens=20000, output_mode='tf_idf')
text_vectorization.adapt(text_only_train_ds)

In [12]:
tfidf_2gram_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)
tfidf_2gram_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)
tfidf_2gram_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)

callback_list = [
    callbacks.ModelCheckpoint('tfidf_2gram', save_best_only=True)
]

model = get_model()
model.fit(tfidf_2gram_train_ds.cache(), validation_data=tfidf_2gram_val_ds, epochs=10, callbacks=callback_list)

model = keras.models.load_model('tfidf_2gram')
print(model.evaluate(tfidf_2gram_test_ds))

Epoch 1/10
616/625 [============================>.] - ETA: 0s - loss: 0.4953 - binary_accuracy: 0.7837INFO:tensorflow:Assets written to: tfidf_2gram/assets


INFO:tensorflow:Assets written to: tfidf_2gram/assets


625/625 [==============================] - 1s 2ms/step - loss: 0.4939 - binary_accuracy: 0.7846 - val_loss: 0.3002 - val_binary_accuracy: 0.9000
Epoch 2/10
612/625 [============================>.] - ETA: 0s - loss: 0.3247 - binary_accuracy: 0.8656INFO:tensorflow:Assets written to: tfidf_2gram/assets


INFO:tensorflow:Assets written to: tfidf_2gram/assets


625/625 [==============================] - 1s 2ms/step - loss: 0.3249 - binary_accuracy: 0.8660 - val_loss: 0.2950 - val_binary_accuracy: 0.8984
Epoch 3/10
625/625 [==============================] - 1s 1ms/step - loss: 0.2810 - binary_accuracy: 0.8880 - val_loss: 0.3147 - val_binary_accuracy: 0.8978
Epoch 4/10
625/625 [==============================] - 1s 1ms/step - loss: 0.2534 - binary_accuracy: 0.8967 - val_loss: 0.3345 - val_binary_accuracy: 0.8986
Epoch 5/10
625/625 [==============================] - 1s 1ms/step - loss: 0.2442 - binary_accuracy: 0.9011 - val_loss: 0.3695 - val_binary_accuracy: 0.8890
Epoch 6/10
625/625 [==============================] - 1s 1ms/step - loss: 0.2297 - binary_accuracy: 0.9069 - val_loss: 0.3443 - val_binary_accuracy: 0.8904
Epoch 7/10
625/625 [==============================] - 1s 1ms/step - loss: 0.2165 - binary_accuracy: 0.9107 - val_loss: 0.3641 - val_binary_accuracy: 0.8958
Epoch 8/10
625/625 [==============================] - 1s 1ms/step - loss: 0

In [14]:
max_length = 600
max_tokens = 20000

text_vectorization = layers.TextVectorization(max_tokens=max_tokens, output_mode='int', output_sequence_length=max_length)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)

In [19]:
import tensorflow as tf

inputs = keras.Input(shape=(None,), dtype=tf.int64)
embedded = tf.one_hot(inputs, depth=max_tokens)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation=activations.sigmoid)(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=optimizers.RMSprop(), loss=losses.BinaryCrossentropy(), metrics=[metrics.BinaryAccuracy()])
model.summary()

callback_list = [
    callbacks.ModelCheckpoint('one_hot_bidir_lstm', save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=1, callbacks=callback_list)

model = keras.models.load_model('one_hot_bidir_lstm')
print(model.evaluate(int_test_ds))

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, None)]            0         
                                                                 
 tf.one_hot_3 (TFOpLambda)   (None, None, 20000)       0         
                                                                 
 bidirectional_3 (Bidirecti  (None, 64)                5128448   
 onal)                                                           
                                                                 
 dropout_6 (Dropout)         (None, 64)                0         
                                                                 
 dense_9 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5128513 (19.56 MB)
Trainable params: 5128513 (19.56 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

INFO:tensorflow:Assets written to: one_hot_bidir_lstm/assets


782/782 [==============================] - 889s 1s/step - loss: 0.3902 - binary_accuracy: 0.8344
[0.39024680852890015, 0.8343600034713745]


In [24]:
inputs = keras.Input(shape=(None,), dtype=tf.int64)
x = tf.one_hot(inputs, depth=max_tokens)
x = layers.SeparableConv1D(filters=64, kernel_size=3, activation=activations.relu)(x)
x = layers.MaxPooling1D(2)(x)
x = layers.SeparableConv1D(filters=64, kernel_size=3, activation=activations.relu)(x)
x = layers.MaxPooling1D(2)(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation=activations.sigmoid)(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=optimizers.RMSprop(), loss=losses.BinaryCrossentropy(), metrics=[metrics.BinaryAccuracy()])
model.summary()

callback_list = [
    callbacks.ModelCheckpoint('one_hot_conv', save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=1, callbacks=callback_list)

model = keras.models.load_model('one_hot_conv')
print(model.evaluate(int_test_ds))

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, None)]            0         
                                                                 
 tf.one_hot_7 (TFOpLambda)   (None, None, 20000)       0         
                                                                 
 separable_conv1d_9 (Separa  (None, None, 64)          1340064   
 bleConv1D)                                                      
                                                                 
 max_pooling1d_9 (MaxPoolin  (None, None, 64)          0         
 g1D)                                                            
                                                                 
 separable_conv1d_10 (Separ  (None, None, 64)          4352      
 ableConv1D)                                                     
                                                          

INFO:tensorflow:Assets written to: one_hot_conv/assets


782/782 [==============================] - 362s 463ms/step - loss: 0.6932 - binary_accuracy: 0.5000
[0.6931540369987488, 0.5]
